In [4]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib, loadJoblib
from webUtils import getWebData, getHTML, removeTag
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getGeos
import pickle
from glob import glob
import geopy
import requests
from os.path import basename,splitext,join
from collections import Counter

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-12-14 09:50:30.051329


# Global Parameters

In [22]:
prec = 7
size = 50 # meters
basedir = "/Users/tgadf/Downloads/gasbuddy"
# http://gas-stations.citgo.com/tn/knoxville

In [19]:
states={}
states["AL"] = "Alabama"
states["AK"] = "Alaska"
states["AZ"] = "Arizona"
states["AR"] = "Arkansas"
states["CA"] = "California"
states["CO"] = "Colorado"
states["CT"] = "Connecticut"
states["DE"] = "Delaware"
states["DC"] = "District of Columbia"
states["FL"] = "Florida"
states["GA"] = "Georgia"
states["ID"] = "Idaho"
states["IL"] = "Illinois"
states["IN"] = "Indiana"
states["IA"] = "Iowa"
states["KS"] = "Kansas"
states["KY"] = "Kentucky"
states["LA"] = "Louisiana"
states["ME"] = "Maine"
states["MD"] = "Maryland"
states["MA"] = "Massachusetts"
states["MI"] = "Michigan"
states["MN"] = "Minnesota"
states["MS"] = "Mississippi"
states["MO"] = "Missouri"
states["MT"] = "Montana"
states["NE"] = "Nebraska"
states["NV"] = "Nevada"
states["NH"] = "New Hampshire"
states["NJ"] = "New Jersey"
states["NM"] = "New Mexico"
states["NY"] = "New York"
states["NC"] = "North Carolina"
states["ND"] = "North Dakota"
states["OH"] = "Ohio"
states["OK"] = "Oklahoma"
states["OR"] = "Oregon"
states["PA"] = "Pennsylvania"
states["RI"] = "Rhode Island"
states["SC"] = "South Carolina"
states["SD"] = "South Dakota"
states["TN"] = "Tennessee"
states["TX"] = "Texas"
states["UT"] = "Utah"
states["VT"] = "Vermont"
states["VA"] = "Virginia"
states["WA"] = "Washington"
states["WV"] = "West Virginia"
states["WI"] = "Wisconsin"
states["WY"] = "Wyoming"
_, _ = clock("Last Run")

Current Time is Fri Dec 14, 2018 12:24:41 for Last Run


In [24]:
for abbr,name in states.items():
    url = "https://www.allgasstations.com/{0}/".format(abbr)
    savename = join(basedir, "{0}.html".format(abbr))
    if not isFile(savename):
        getWebData(base=url, suburl=None, dtime=5, useSafari=False, savename=savename)

HTTPError: HTTP Error 403: Forbidden

In [ ]:
from time import sleep
for abbr,name in states.items():
    filename = "/Users/tgadf/Downloads/ags/{0}.html".format(abbr)
    bsdata   = getHTML(loadJoblib(filename))
    for h2 in bsdata.findAll("a"):        
        attrs = h2.attrs
        href  = attrs['href']
        if not href.startswith("/"):
            continue
        url = "https://www.allgasstations.com/{0}".format(href)
        savename = "/Users/tgadf/Downloads/ags/indiv/{0}.html".format(href[1:-1].replace("/", "_"))
        if not isFile(savename):
            getWebData(base=url, suburl=None, dtime=5, useSafari=False, savename=savename)
            sleep(0.33)
        
    #print(bsdata)

In [ ]:
from glob import glob
addresses={}
for abbr,name in states.items():
    files = glob("/Users/tgadf/Downloads/ags/indiv/{0}_*.html".format(abbr))
    addresses[abbr] = []
    for filename in files:
        bsdata = getHTML(loadJoblib(filename))
        for i, div in enumerate(bsdata.findAll("div")):
            if div.attrs.get('class') is None:
                continue
            if 'col' in div.attrs['class']:
                for d in div.findAll("div"):
                    tmp = removeTag(line=d, tag='a')
                    tmp = removeTag(line=tmp, tag='i')                    
                    address = tmp.text
                    addresses[abbr].append(address)
    print("Found {0} stations in {1}".format(len(addresses[abbr]), name))

In [ ]:
saveJoblib(addresses, "/Users/tgadf/Downloads/ags/stations.p")

# 1st Pass for Locations

In [ ]:
from geopy.geocoders import Nominatim
from time import sleep
geolocator = Nominatim(user_agent="tgadf")
addresses  = loadJoblib("/Users/tgadf/Downloads/ags/stations.p")
results = {}
for abbr,lookups in addresses.items():
    results[abbr] = {}
    for la, address in enumerate(lookups):
        savename = "/Users/tgadf/Downloads/ags/{0}_{1}_results.p".format(abbr,la)
        if isFile(savename):
            continue
        vals = address.split("\n")
        for iv,val in enumerate(vals):
            if len(val) < 3:
                continue
            print("--> {0}/{1}\t{2}/{3}\t{4}".format(la+1,len(lookups),iv+1,len(vals),val))
            try:
                location = geolocator.geocode(val)
            except:
                continue
            try:
                print(location.address,location.latitude, location.longitude)
            except:
                continue
            results[abbr][val] = [location.latitude, location.longitude]
            sleep(1)
            
        saveJoblib(results, savename)

In [ ]:
val = "919 East Dimond Boulevard Anchorage, AK 99515"
location = geolocator.geocode(val)
print(location.address)
print((location.latitude, location.longitude))

# Check For Missing

In [7]:
from geopy.geocoders import Nominatim
from time import sleep
geolocator = Nominatim(user_agent="tgadf")
addresses  = loadJoblib("/Users/tgadf/Downloads/ags/stations.p")
results = {}
for abbr,lookups in addresses.items():
    print("===> {0}".format(abbr))
    results[abbr] = {}
    for la, address in enumerate(lookups):
        savename = "/Users/tgadf/Downloads/ags/{0}_{1}_results.p".format(abbr,la)
        resave   = False
        if isFile(savename):
            results = loadJoblib(savename)
        else:
            continue
                
        vals = address.split("\n")
        for iv,val in enumerate(vals):
            if len(val) < 3:
                continue
                
            if results[abbr].get(val) is None:
                print("--> {0}/{1}\t{2}/{3}\t{4}".format(la+1,len(lookups),iv+1,len(vals),val))
                try:
                    val = val.replace("USHighway ", "US")
                    location = geolocator.geocode(val)
                    print(location.address,location.latitude, location.longitude)
                    resave = True
                except:
                    continue
                results[abbr][val] = [location.latitude, location.longitude]
                sleep(1)

        if resave == True:
            print("Resaving {0}".format(savename))
            saveJoblib(results, savename)

# Merge Locations

In [10]:
locations = {}
from glob import glob
for abbr,statename in states.items():
    files = glob("/Users/tgadf/Downloads/ags/{0}_*_results.p".format(abbr))
    for filename in files:
        if isFile(filename):
            results = loadJoblib(filename)
        else:
            raise ValueError("Could not open {0}".format(filename))
            
        for state,values in results.items():
            for name,location in values.items():
                locations[name] = location
    print(len(locations),'\t',statename)

savename = "/Users/tgadf/Downloads/ags/locations.p"
saveJoblib(locations, savename)

624 	 Alabama
685 	 Alaska
1108 	 Arizona
1416 	 Arkansas
4591 	 California
5052 	 Colorado
5748 	 Connecticut
5838 	 Delaware
5921 	 District of Columbia
7227 	 Florida
7858 	 Georgia
7939 	 Idaho
9352 	 Illinois
9938 	 Indiana
10342 	 Iowa
10749 	 Kansas
11239 	 Kentucky
11979 	 Louisiana
12223 	 Maine
13153 	 Maryland
14294 	 Massachusetts
15677 	 Michigan
16238 	 Minnesota
16573 	 Mississippi
17170 	 Missouri
17284 	 Montana
17565 	 Nebraska
17723 	 Nevada
17938 	 New Hampshire
18991 	 New Jersey
19262 	 New Mexico
20757 	 New York
21696 	 North Carolina
21839 	 North Dakota
23042 	 Ohio
23425 	 Oklahoma
23913 	 Oregon
24905 	 Pennsylvania
25088 	 Rhode Island
25563 	 South Carolina
25685 	 South Dakota
26539 	 Tennessee
28211 	 Texas
28393 	 Utah
28531 	 Vermont
29536 	 Virginia
30314 	 Washington
30559 	 West Virginia
31461 	 Wisconsin
31555 	 Wyoming
  --> This file is 1.1MB.


# Find Geohashs

In [15]:
filename = "/Users/tgadf/Downloads/ags/locations.p"
locations = loadJoblib(filename)

geomap = {}
nl   = 0
for name,location in locations.items():
    lat0 = location[0]
    lng0 = location[1]
    
    dLat = convertMetersToLat(size)
    dLng = convertMetersToLong(size, lat0)
    nl  += 1
    
    for i in [1, 0, -1]:
        for j in [1, 0, -1]:
            lat = lat0 + i*dLat
            lng = lng0 + j*dLng
            
            geo = geohash.encode(latitude=lat, longitude=lng, precision=prec)
            geomap[geo] = {"AGSFuel": 1}
            
    for geo in geomap.keys():
        geomap[geo] = dict(geomap[geo])            
            
            
    if nl % 1000 == 0:
        print("Found {0} locations and {1} geos".format(nl, len(geomap)))


print("Found {0} locations and {1} geos".format(nl, len(geos)))
fname="/Users/tgadf/Downloads/ags/geomap-{0}-AGS.p".format(prec)
print("Writing {0}".format(fname))
saveJoblib(geomap, fname)

Found 1000 locations and 2798 geos
Found 2000 locations and 5546 geos
Found 3000 locations and 8265 geos
Found 4000 locations and 10959 geos
Found 5000 locations and 13680 geos
Found 6000 locations and 16454 geos
Found 7000 locations and 19081 geos
Found 8000 locations and 21766 geos
Found 9000 locations and 24640 geos
Found 10000 locations and 27500 geos
Found 11000 locations and 30408 geos
Found 12000 locations and 33111 geos
Found 13000 locations and 35752 geos
Found 14000 locations and 38510 geos
Found 15000 locations and 41310 geos
Found 16000 locations and 44174 geos
Found 17000 locations and 47014 geos
Found 18000 locations and 49956 geos
Found 19000 locations and 52794 geos
Found 20000 locations and 55569 geos
Found 21000 locations and 58345 geos
Found 22000 locations and 61256 geos
Found 23000 locations and 63966 geos
Found 24000 locations and 66811 geos
Found 25000 locations and 69662 geos
Found 26000 locations and 72453 geos
Found 27000 locations and 75199 geos
Found 28000 l

# GeoSpatial Plots

In [ ]:
# Load (if needed)
# geos = loadJoblib("/Users/tgadf/Downloads/ags/geomap-{0}-AGS.p".format(prec)).keys()

In [17]:
from folium import PolyLine, CircleMarker, Circle, Marker, Icon, FeatureGroup, Map, LayerControl
import geohash

def addCircles(geos, m, init=None, maxDist=None, debug=False):
    cols = ['darkgreen']

    toPlot = []
    for geo in geos:
        com = geohash.decode_exactly(geo)[:2]
        if init is not None:
            if getDist(com, init) < maxDist:
                toPlot = getBB(geo, istuple=True)
                PolyLine(toPlot, color='blue', weight=1, opacity=1).add_to(m)
        else:
            Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=1, opacity=0).add_to(m)
        
def createMap(init=None):
    if init is not None:
        m = Map(location=[init[0], init[1]], zoom_start=10)
    else:
        m = Map(location=[40, -95], zoom_start=4)
        
    return m        

In [18]:
init = init=(36, -84)
m = createMap(init)
addCircles(geos, m=m, init=init, maxDist=100000)
m.save("/Users/tgadf/Downloads/ags/ags-map.html")